# 🚘 Analyzing SF traffic stops with `R`: Part 3

<img src="img/sf-traffic-3.jpg" alt="traffic" width="400" align="left"/>

This is Part 3. Other parts can be found [here](https://www.jdgrossman.com).

## Introduction

In this series of tutorials, we'll use `R` to explore traffic stops in San Francisco (SF). In particular, we'll investigate whether there is evidence of racial discrimination in SF's policing practices. 

> **Important note**: Policing can be a sensitive subject. It's important to remember that each row in our data represents a real interaction between a police officer and driver. Please keep this in mind as you work through the tutorial, and be sure to engage with the material to the extent you're comfortable. 

By the end of the tutorials, you'll have foundational understanding of the following:
1. 📊 How to use `R` to explore tabular data and calculate descriptive statistics. 
2. 📈 How to make an informative plot with `R`
2. ⚖️ How to approach questions about social policy with data. 

Let's get started!

## ✅ Set up

While the core `R` language contains many useful functions (e.g., `sum` and `sample`), there is vast functionality built on top of `R` by community members.

Make sure to run the cell below. It imports additional useful functions, adjusts `R` settings, and loads in data. 

In [0]:
# Load in additional functions
library(tidyverse)
library(lubridate)

# Use three digits past the decimal point
options(digits = 3)

# This makes our plots look nice!
theme_set(theme_bw())

# This is where the data is stored.
STOPS_PATH = "https://github.com/joshuagrossman/dsb-win-2023/raw/main/opp-munging-plotting/data/sf_stop_data.rds"
POP_PATH = "https://github.com/joshuagrossman/dsb-win-2023/raw/main/opp-munging-plotting/data/sf_pop_2015.rds"

# Read in the data
stops <- read_rds(STOPS_PATH)
pop_2015 <- read_rds(POP_PATH)

# Data modifications from Part 1 and
# additional modification for learning purposes. 
stops_w_yr = stops %>% 
  mutate(
    yr = year(date)
  )

pop_2015 = pop_2015 %>%
  mutate(prop_residents = n_people/sum(n_people))

## 📝 Interpreting racial disparities

In Part 1, we counted the number of stops among drivers each race and ethnicity group for the year 2015.

In [0]:
stops_2015 = stops_w_yr %>%
  filter(yr==2015)

# Note that this code is the same as count(stops_2015, race)
stops_2015 %>%
  group_by(race) %>%
  summarize(n = n()) %>%
  ungroup()

For ease of interpretation, we can add a column with the proportion of all stops that occurred among drivers of each race group:

In [0]:
stop_counts = stops_2015 %>%
  group_by(race) %>%
  summarize(
    n = n()
  ) %>%
  ungroup() %>%
  mutate(
    prop_stops = n/sum(n)
  )

stop_counts

### 💬 Discussion

What can we conclude from the table above? 

What are the limitations of this table?

Is there information that could make this table more informative? 

### Double click here to write your thoughts.
\
\
\
\
\
\

## 📝 Incorporating additional data

With the data above, we can claim that most stops in San Francisco in 2015 occurred among white drivers, followed by Asian/Pacific Islander drivers, Black drivers, drivers of an unlisted race/ethnicity, and Hispanic drivers.

However, without knowing the demographics of drivers in San Francisco, we can't say anything about racial disparities in stop rates.

> For example, if 20% of drivers in San Francisco were white, and 35% of stops were among white drivers, we might claim that white drivers were stopped more frequently than expected.

Fortunately, the demographics of San Francisco residents in 2015 are provided in the `pop_2015` data:

In [0]:
pop_2015

With the table above, we have more context!

For example, while 5% of San Francisco identifies as Black, nearly 21% of stops occurred among Black drivers. 

### 💬 Discussion

Using only the data above, can we claim that Black drivers in SF were stopped on the basis of less evidence than drivers of other race groups? Do your conclusions depend on any assumptions?

### Double click here to write your thoughts.
\
\
\
\
\
\

## 📝 Merging data

Suppose we are interested in the ratio of the stop rate for each race/ethnicity group and the fraction of residents from each race/ethnicity group.

We could do this by hand, but it's a little tedious.

In [0]:
print('Asian/Pacific Islander drivers: stop rate / pop. rate:')
0.181/0.3393

print('Black drivers: stop rate / pop. rate:')
0.175/0.0534

print('Hispanic drivers: stop rate / pop. rate:')
0.139/0.1530

print('White drivers: stop rate / pop. rate:')
0.347/0.4124

print('Drivers of other race/ethnicity: stop rate / pop.rate:')
0.158/0.0419

It would be a lot faster if we had the stop rates and pop. rates in one table.

### 🧑‍🤝‍🧑 Joining data with shared columns

Imagine writing our two tables side by side. Here's a (bad!!) way to do it:

In [0]:
# Reminder: this is generally NOT a good thing to do to merge data.
# We're doing this just for the visual!
bind_cols(stop_counts, pop_2015)

☝️ There's a big problem with the table above: the race columns are not aligned!

> For example, Asian/Pacific Islander driver stop rates are next to white driver pop. rates. This is not good!

Better idea: what if we could combine each row of our tables by making sure the value in the race column is the same in both rows?

Here's how to do it:

In [0]:
combined_data = left_join(stop_counts, pop_2015, by='race')

combined_data

The next section will break down this code.

### 💫 The magic of `left_join`

`left_join()` only needs three things from you:
1. What's the first table?
2. What's the second table?
3. Which column should match?

🖥️ Usage: `left_join(table1, table2, by='column_name')`
* `table1`: the first table (i.e., 'left table')
* `table2`: the second table (i.e., 'right table')
* `column_name`: the name of the column to match

But, we've left one question unanswered: why `left_join()` and not just `join()`?

Here's a visual explanation.

> Note: `NA` indicates a missing value that is "not available".

<img src="img/left_join_updated.png" alt="inner join" width="400" align="left"/>

`left_join()` always returns every row in the **left** table. 

- However, `left_join()` only includes rows from the **right** table that can be matched to rows in the **left** table.

- The matched column is often referred to as the "join key". That's why it's named `key` in the picture.

- Unmatched rows from the **right** table are dropped.

Here's the picture in code form. Play around with the code to get a feel for `left_join()`!

In [0]:
table1 = tibble(
    val_x = c('x1', 'x2', 'x3'),
    key = c(1, 2, 3)
)

table2 = tibble(
    val_y = c('y1', 'y2', 'y3'),
    key = c(1, 2, 4)
)

table1
table2
left_join(table1, table2, by='key')

### 🙀 `right_join()`, `inner_join()`, `anti_join()`, [oh my](https://www.youtube.com/watch?v=-HrfbV16-FQ)!

There are actually a few more joining functions. But, **you only need left join for >99% of data science work**. 

#### 🚀 Mini-exercise

Run the three joining functions in the cell below. 

> The arguments of `right_join()`, `inner_join()`, and `anti_join()` are the same as `left_join()` above.

What do these three new joining functions do?

In [0]:
table1 = tibble(
    val_x = c('x1', 'x2', 'x3'),
    key = c(1, 2, 3)
)

table2 = tibble(
    val_y = c('y1', 'y2', 'y3'),
    key = c(1, 2, 4)
)

# Your code here!



## 🚀 Exercise: Putting it all together

In this exercise, you may find it helpful to refer to code from Parts 1 and 2. You can find Parts 1 and 2 [here](https://www.jdgrossman.com).

#### Part A

Calculate the proportion of all stops, searches, and contraband detections that occurred for SF drivers of each race and ethnicity group in 2015.

What do you conclude from your findings? How could someone use these results to mislead their audience?

In [0]:
# Your code here!

# As a reminder, here's how we did this for stops:
stop_counts = stops_2015 %>%
  group_by(race) %>%
  summarize(n = n()) %>%
  ungroup()



#### Part B

For each race and ethnicity group, calculate the ratio of stop rates, search rates, and contraband detection rates to the population rate for 2015.

What do you conclude from your findings? How could someone use these results to mislead their audience?

In [0]:
# Your code here!



#### Part C

First, filter the `stops` data to drivers who identify as Black. 

Second, for Black drivers in each district, calculate the fraction of stops that resulted in a search ("search rate"). Store your result in a dataframe.

Repeat the first two steps for all race/ethnicity groups in the data. 

In [0]:
# Your code here!



❗❗Important note❗❗: You are encouraged to repeat Parts D, E, and F using different pairs of race/ethnicity groups.

#### Part D

For each district, calculate the difference in search rates of Black drivers and white drivers. 

> This will require you to merge two dataframes that you created in Part C.

What do you conclude from your findings? How could you present this result concisely to a non-technical audience?

In [0]:
# Your code here!



#### Part E

Using your merged data from Part D, create a plot with the following features:
- On the x-axis, plot the search rate for Black drivers.
- On the y-axis, plot the search rate for white drivers.
- Color the points by district
- Add (`+`) the following lines to your plotting code: 

```
geom_abline(linetype = 'dashed') +
xlim(0, 0.5) +
ylim(0, 0.5)
```

What do you conclude from your findings?

In [0]:
# Your code here!



#### Part F

Repeat Parts D and E, but instead of calculating search rates, calculate the proportion of searches that resulted in contraband being found ("hit rate").

What do you conclude from your findings?

In [0]:
# Your code here!



## Concluding remarks

In the final exercise of this tutorial, you conducted an **outcome test**. 

> Figure 3 in [this paper](https://5harad.com/papers/100M-stops.pdf) shows the result of an outcome test applied to large-scale policing data. 

Outcome tests can be used to detect potential discrimination not just in policing.
For example, think about how you might use an outcome test to detect discrimination in bank lending or hiring decisions.